<a href="https://colab.research.google.com/github/ketankishore27/TensorFlow/blob/master/Data_Validation_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
python-snappy is already the newest version (0.5-1.1build2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [0]:
!pip install -q tensorflow_data_validation

     |████████████████████████████████| 2.4MB 2.8MB/s 
     |████████████████████████████████| 2.8MB 42.7MB/s 
     |████████████████████████████████| 51kB 20.0MB/s 
     |████████████████████████████████| 1.0MB 33.1MB/s 
     |████████████████████████████████| 143kB 39.7MB/s 
ERROR: multiprocess 0.70.8 has requirement dill>=0.3.0, but you'll have dill 0.2.9 which is incompatible.


In [0]:
import pandas as pd
import tensorflow_data_validation as tfdv
import tensorflow as tf
import copy
from __future__ import print_function

In [0]:
dataset = pd.read_csv('pollution-small.csv')

In [0]:
training_data = dataset[:1600]

In [0]:
test_set = dataset[1600:]

In [0]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe = training_data)

In [0]:
schema = tfdv.infer_schema(statistics = train_stats)

In [0]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'soot',FLOAT,required,,-
'no2',FLOAT,required,,-
'pm10',FLOAT,required,,-
'Date',BYTES,required,,-
'so2',FLOAT,required,,-


In [0]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe = test_set)

In [0]:
anamolies = tfdv.validate_statistics(statistics = test_stats, schema = schema)

In [0]:
tfdv.display_anomalies(anamolies)

In [0]:
test_set.drop('soot', axis = 1, inplace = True)

/usr/local/lib/python2.7/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe = test_set)

In [0]:
anamolies = tfdv.validate_statistics(statistics = test_stats, schema = schema)

In [0]:
tfdv.display_anomalies(anamolies)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


In [0]:
tfdv.visualize_statistics(test_stats)

In [0]:
schema.default_environment.append('training')
schema.default_environment.append('serving')

In [0]:
tfdv.get_feature(schema = schema, feature_path = 'soot').not_in_environment.append('serving')

In [0]:
serving_env_anamolies = tfdv.validate_statistics(statistics = test_stats, schema = schema, environment = 'serving')

In [0]:
tfdv.display_anomalies(serving_env_anamolies)

In [0]:
tfdv.write_schema_text(schema = schema, output_path = 'pollution_schema.pbtxt')

In [0]:
schema = tfdv.load_schema_text(input_path = 'pollution_schema.pbtxt')

In [0]:
tfdv.visualize_statistics(train_stats, test_stats)